# ETL Processes

In [43]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

**Database Connection**

In [44]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=2105")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres Database")
    print(e) 

**Create Cursor**

In [45]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the Postgres Database")
    print(e)

**Set Autocommit to our transactions**

In [46]:
conn.set_session(autocommit=True)

**Get file path**

In [47]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, "*.json"))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

## Process song-data

In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [48]:
song_files = get_files('data/song_data')

In [49]:
filepath = song_files[1]
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969


## Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

## Songs Table

In [50]:
df.values[0].tolist()

[1,
 'ARMJAGH1187FB546F3',
 35.14968,
 -90.04892,
 'Memphis, TN',
 'The Box Tops',
 'SOCIWDW12A8C13D406',
 'Soul Deep',
 148.03546,
 1969]

In [51]:
values = {}
for column in df.columns:
    values[column] = df[column].values[0]

print(values)

{'num_songs': 1, 'artist_id': 'ARMJAGH1187FB546F3', 'artist_latitude': 35.14968, 'artist_longitude': -90.04892, 'artist_location': 'Memphis, TN', 'artist_name': 'The Box Tops', 'song_id': 'SOCIWDW12A8C13D406', 'title': 'Soul Deep', 'duration': 148.03546, 'year': 1969}


In [52]:
for keys, v in values.items():
    print(v, type(v))

1 <class 'numpy.int64'>
ARMJAGH1187FB546F3 <class 'str'>
35.14968 <class 'numpy.float64'>
-90.04892 <class 'numpy.float64'>
Memphis, TN <class 'str'>
The Box Tops <class 'str'>
SOCIWDW12A8C13D406 <class 'str'>
Soul Deep <class 'str'>
148.03546 <class 'numpy.float64'>
1969 <class 'numpy.int64'>


In [53]:
song_data = [values['song_id'], values['title'], values['artist_id'], int(values['year']), float(values['duration'])]
song_data

['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546]

### Insert Record into Song Table

Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [54]:

cur.execute(songs_table_insert, song_data)
conn.commit()

## Artists Table

- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [55]:
artist_data = [values['artist_id'], values['artist_name'], values['artist_location'], float(values['artist_longitude']), float(values['artist_latitude'])]
artist_data

['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', -90.04892, 35.14968]

### Insert Record into Artist Table

Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [56]:
cur.execute(artists_table_insert, artist_data)
conn.commit()

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [57]:
log_files = get_files('data/log_data')

In [58]:
filepath = log_files[0]
filepath

'c:\\Users\\user\\Documents\\Tutorials\\data science\\data science portfolio\\data_modelling_with_postgres\\data\\log_data\\2018\\11\\2018-11-01-events.json'

In [59]:
df = pd.read_json(filepath, lines = True)
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         11 non-null     object 
 1   auth           15 non-null     object 
 2   firstName      15 non-null     object 
 3   gender         15 non-null     object 
 4   itemInSession  15 non-null     int64  
 5   lastName       15 non-null     object 
 6   length         11 non-null     float64
 7   level          15 non-null     object 
 8   location       15 non-null     object 
 9   method         15 non-null     object 
 10  page           15 non-null     object 
 11  registration   15 non-null     int64  
 12  sessionId      15 non-null     int64  
 13  song           11 non-null     object 
 14  status         15 non-null     int64  
 15  ts             15 non-null     int64  
 16  userAgent      15 non-null     object 
 17  userId         15 non-null     int64  
dtypes: float64(1

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## Time Table

#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [60]:
df = df[df['page'] == 'NextSong']
df.shape

(11, 18)

In [61]:
t = pd.to_datetime(df['ts'], unit='ms') 
t.head()

2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
5   2018-11-01 21:08:16.796
6   2018-11-01 21:11:13.796
7   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [62]:
time_data = []
for line in t:
    time_data.append([line, 
                      line.hour, 
                      line.day, 
                      line.week, 
                      line.month, 
                      line.year, 
                      line.day_name()])
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [63]:
time_df = pd.DataFrame.from_records(time_data, columns=column_labels)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,Thursday
1,2018-11-01 21:05:52.796,21,1,44,11,2018,Thursday
2,2018-11-01 21:08:16.796,21,1,44,11,2018,Thursday
3,2018-11-01 21:11:13.796,21,1,44,11,2018,Thursday
4,2018-11-01 21:17:33.796,21,1,44,11,2018,Thursday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [64]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row)
    conn.commit()
    #print(i,list(row))

## Users Table

#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [65]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [66]:
for i, row in user_df.iterrows():
    cur.execute(users_table_insert, row)
    conn.commit()

## song_plays Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [69]:
song_select = """
    SELECT song_id, artists.artist_id
    FROM songs JOIN artists ON songs.artist_id = artists.artist_id
    WHERE songs.title = %s
    AND artists.name = %s
    AND songs.duration = %s
"""

for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ( pd.to_datetime(row.ts, unit='ms'), 
                     int(row.userId), 
                     row.level, 
                     songid, 
                     artistid, 
                     row.sessionId, 
                     row.location, 
                     row.userAgent)
    
    cur.execute(songplays_table_insert, songplay_data)
    conn.commit()

**Close Connection to Sparkify DB**

In [70]:
conn.close()